In [4]:
import numpy as np
import scipy
import qiskit
from qiskit import ClassicalRegister,QuantumRegister,QuantumCircuit,execute
from qiskit.extensions import UnitaryGate
from qiskit.circuit.library import PhaseEstimation

# [Quantum Recommendation System](https://arxiv.org/abs/1603.08675)

Quantum Recommendation System (QRS) is a quantum algorithm that provides personalized recommendation to individual users given their partial information. The information is given as an $m \times n$ preference matrix, which is assumed to have a good rank-$k$ approximation. It is the first algorithm to have polylogarithmic runtime in dimensions $mn$, $O(\text{poly}(k)\text{polylog}(mn))$. The advantage comes from an efficient sampling from an approximation of the preference matrix without reconstructing the entire matrix. This provides one of the first examples of application of quantum machine learning in real-world problems.

## Data Structure
For a matrix $A \in \mathbb{R}^{m \times n}$, $m$ is number of users and $n$ is number of objects. The data structure consists of an array of $m$ binary trees, each representing a user. The leaves store the value $A_{ij}^2$ as well as the sign of $A_{ij}$. The depth of each tree is at most $\lceil \log_2 n\rceil$. An internal node $v$ of tree stores the sum of the values of all leaves in the subtree rooted. 
When a new entry $(i,j,A_{ij})$ arrives the left node $j$ in tree is created if not presented, all the nodes on the path from that leaf to the tree root are also updated. 


In [11]:
class Node: # Structure of a node 
    def __init__(self,data): 
        self.data = data
        self.left = None
        self.right = None
        
    def updateNode(self,data):
        self.data = data

In [63]:
class FullBinaryTree:
    def __init__(self,row):
        self.row = row
        self.num_leaves = len(self.row)
        assert (self.num_leaves & (self.num_leaves-1) == 0) and self.num_leaves != 0, "Data is expected to have the length of a power of 2"
        
        self.height = int(np.log2(self.num_leaves)) + 1
        self.root = self.createTree()
        self.insertLeaves(self.root,0)
        self.fillTree(self.root)
                
            
    def createTree(self): 
        """
        Create a full binary tree corresponding to the given row
        """
        ## Helper method
        def createTree(cur_root,node_idx):
            if node_idx < non_leave_nodes:
                if node_idx > 0:
                    cur_root = Node(data=0)         
                cur_root.left = createTree(cur_root.left, 2*node_idx+1)
                cur_root.right = createTree(cur_root.right, 2*node_idx+2)            
            return cur_root
            
        root = Node(data=0)     
        non_leave_nodes = self.num_leaves - 1 # For a complete binary tree, #inner nodes = #leaves - 1       
        createTree(root,0)
        return root

    def computeLeaveValue(self,value):
        return np.sign(value)*(value**2)
    
    def insertLeaves(self,cur_root,leave_idx):
        """
        Assign data for leaves
        """
        if (not cur_root):
            return

        if cur_root.left:
            leave_idx = self.insertLeaves(cur_root.left,leave_idx)
        else:
            if leave_idx < len(self.row):
                cur_root.left = Node(self.computeLeaveValue(self.row[leave_idx]))
                leave_idx += 1

        if cur_root.right:
            leave_idx = self.insertLeaves(cur_root.right,leave_idx)
        else:
            if leave_idx < len(self.row):
                cur_root.right = Node(self.computeLeaveValue(self.row[leave_idx]))
                leave_idx += 1
        return leave_idx    
    
    def fillTree(self,root): # Calculating the inner node data and fill the tree with data, the root data is equal the sum of all leaves data
        if not root:
            return

        if root.left and root.right:
            if not (root.left.left and root.left.right and root.right.left and root.right.right):
                root.data = abs(root.left.data) + abs(root.right.data)
            else:
                self.fillTree(root.left)
                self.fillTree(root.right)
                root.data = abs(root.left.data) + abs(root.right.data)    

                
    def getRoot(self):
        return self.root
    
    def updateLeave(self,leave_idx,val):
        root = self.root
        max_level = self.height - 1       
        left = 0
        midpoint = 2**max_level / 2
        
        while leave_idx != midpoint:
            if leave_idx < midpoint + left:
                root = root.left
            else:
                root = root.right
                left = midpoint + left
            midpoint = int(midpoint/2)
        root.data = val
        return

In [74]:
rowdata = [1,-2,3,4,-5,6,-10,8]
rowtree = FullBinaryTree(rowdata)

In [75]:
root = rowtree.getRoot()

In [73]:
root.right.right.left.data

-49

## Preference matrix
From any matrix that records user behaviors $[A]_{ij} \in \mathbb{R}^{m \times n}$, one can construct a binary preference matrix $[T]_{ij}$ by being selective for high values of $A$. A subsample matrix $\hat{T}$ with probability $p$ is created with $\hat{T}_{ij} = T_{ij}/p$ with probability $p$ or $\hat{T}_{ij} = 0$ otherwise. The assumption that $A$ has a good rank-$k$ approximation also applies for $T$ and $\hat{T}$. The property means that the matrix is close to a matrix of rank $k$ with respect to the Frobenius norm: $\left \| T-T_k \right \| < \varepsilon \left \| T \right \|_F$ for some small $\varepsilon > 0$. The main algorithm will take $\hat{T}$ as input.

## Idea
Given $\hat{T} \in \mathbb{R}^{m \times n}$ stored in an efficient data structure and a vector $x \in \mathbb{R}^m$ that represents a user, the algorithm projects $x$ onto the subspace spanned by the union of row singular vectors whose corresponding singular values are greater than $\sigma$ and some subset of row singular vectors whose corresponding singular values are in the interval $[(1-\kappa)\sigma,\sigma)$. Here we choose $\kappa = 1/3$ as a constant and $\sigma = \sqrt{\frac{\varepsilon^2 p}{2k}}\left \| \hat{T} \right \|_F$

In [77]:
def amplitude_encoding(data_vec):
    """
    Encode |x⟩ = x_0|0⟩ + x_1|1⟩ + ... + x_{n-1}|n-1⟩ from initial state |0⟩ for an n-dimension real vector x
    
    Args:
        data_vec: np.array
        
    Return a quantum gate that acts as the encoding unitary operator.
    """
    desired_vector = data_vec[:].astype(np.float64)
    desired_vector = desired_vector / np.linalg.norm(desired_vector)
    
    initializer = qiskit.extensions.Initialize(desired_vector)
    encoding_circ = initializer.gates_to_uncompute().decompose().decompose().inverse().to_gate(label='amplitude_encoder')
    return encoding_circ

In [78]:
def SVE_helper(A):
    """
    Compute helper matrices for the algorithm (see Lemma 5.3)
    
    Args:
        A: np.array (m x n)
    
    Return:
        P: np.array (mn x m)
        Q: np.array (mn x n)
        U: np.array (mn x mn)
        V: np.array (mn x mn)
        W: np.array (mn x mn)
        V_tilda: np.array (mn x mn)
    """
    m,n = A.shape

    A_tilda = np.array([np.linalg.norm(row) for row in A]) # store norm of every row
    norm_A = np.linalg.norm(A)
    
    P = np.zeros((m*n,m))
    for i in range(m):
        unit_vec = np.zeros(m)
        unit_vec[i] = 1
        P[:,i] = np.kron(unit_vec , A[i] / A_tilda[i] )
#         print(unit_vec, A[i] / A_tilda[i])
#         print(P)

    Q = np.zeros((m*n,n))
    for i in range(n):
        unit_vec = np.zeros(n)
        unit_vec[i] = 1
        Q[:,i] = np.kron(A_tilda / norm_A , unit_vec)
#         print(A_tilda / norm_A, unit_vec)
#         print(Q)
        
    
    # Define reflection U,V
    U = 2*P @ P.transpose() - np.identity(m*n)
    V = 2*Q @ Q.transpose() - np.identity(m*n)
        
    # Unitary acting on R[mn]
    W = U @ V
    
    # Basis for {|0^logm, x> for all x in R^m}
    basis_R0 = np.zeros((m*n,n))
    for i in range(n):
        basis_R0[i,i] = 1
    proj_R0 = basis_R0 @ np.linalg.inv((basis_R0.transpose() @ basis_R0)) @ basis_R0.transpose()
    R_0 = 2*proj_R0 - np.identity(m*n)
#    print("basis =",basis_R0)
#    print("R_0",R_0)

    # Diagonalize V
    eigvals,eigvecs = scipy.linalg.schur(V) # More accurate since V is normal
    
    eigvals = np.diag(eigvals)
    idx = eigvals.argsort()[::-1] #large --> small
    eigvals = eigvals[idx]
    eigvecs = eigvecs[:,idx]
    
    V_tilda = eigvecs
    
    return P,Q,U,V,W,V_tilda

In [121]:
def compute_SVE(counts,est_qubits):
    """
    Compute singular values and encode them as binary numbers from the measurement statistics
    
    Args:
        counts: dict{str:str}
        est_qubits: int
        
    Return a dictionary that contains (vector:sv), both in big-endian format
    """
    est_upper = 2**est_qubits
    phase_dict = {}
    shots = sum(counts.values())
    
    # Filter and get phase with highest probability
    for state,freq in counts.items():
        est,data = state.split()        
        if (data not in phase_dict):
            phase_dict[data] = (est,freq/shots)
        elif (freq/shots > phase_dict[data][1]):
            phase_dict[data] = (est,freq/shots)
    print(phase_dict)

    ## Reverse bit order of measured vectors and measured phases
    phase_dict = {vector[::-1]: est[::-1] for vector,(est,freq) in phase_dict.items()} 
    print(phase_dict)
    
    ## Map [0,1] to [-1,1] (for later [-pi,pi])
    ## Note that qiskit returns θ in exp(2iπθ) (θ = measure/2^est_qubits), but need θ in exp(iθ)
    bin_phase_to_num = lambda bitstring: int(bitstring,2)
    map_phase_range = lambda phase: 2*np.pi*phase if phase <= 1./2 else 2*np.pi*phase - 2*np.pi           
    phase_dict = {vector: map_phase_range(bin_phase_to_num(est)/est_upper) for vector,est in phase_dict.items()}
    print("True numerical phase = {}".format(phase_dict))
    
    ## Compute sigma_i = cos(phase/2) and map to bitstring
    phase_to_sv = lambda phase: np.cos(phase/2) * (est_upper) # +1, then *2^n / 2 to map [-1,1] to [0,2] to bitstring(n)
    sv_bound = lambda sv: min(sv,est_upper-1)
    sv_to_bin_sv = lambda sv: bin(int(np.round(sv,decimals=0)))[2:].zfill(est_qubits) # Encode a singular value as the nearest bitstring
    test = {vector: phase_to_sv(phase) for vector,phase in phase_dict.items()}
    print(test)
    sv_dict = {vector: sv_to_bin_sv(sv_bound(phase_to_sv(phase))) for vector,phase in phase_dict.items()} 
    
    print(sv_dict)
    return sv_dict
            

In [128]:
def quantum_SVE(circ,x,A,est_qubits):
    """
    Perform quantum singular value estimation: |x⟩ = sum_{i = 0 to n-1} a_i|v_i⟩ ---> sum_{i = 0 to n-1} a_i|v_i⟩|sigma_i⟩,
    where sigma_i is the corresponding estimate singular values (See Algorithm 5.1)
    
    Args:
        circ: qiskit.circuit.QuantumCircuit
        x: np.array (n)
        A: np.array (m x n)
        est_qubits: int
    """
    P,Q,U,V,W,V_tilda = SVE_helper(A)
    
    m,n = A.shape
    row_norm_reg = QuantumRegister(np.log2(m),name='row_norm')
    circ.add_register(row_norm_reg)

    # 2. Append a first register |0^log(m)> and create the state |Qx>
    Q_gate = UnitaryGate(V_tilda, label='Q gate')
    circ.append(Q_gate, qargs= circ.qregs[0][::-1] + row_norm_reg[::-1]) # qargs = row_norm + data
    
    
    # 3. QPE for unitary W 
    est_reg = QuantumRegister(est_qubits,name='estimate')
    circ.add_register(est_reg)
    
    sv_circ = circ.copy() # Copy the circuit for quantum state sum_{j in [n]} x_j |Q*j>|0>. Used at step 5
    
    W_gate = UnitaryGate(W, label='W gate')
    QPE_circ = PhaseEstimation(est_qubits, W_gate)
    #circ.append(QPE_circ, qargs= est_reg[:] + row_norm_reg[:] + circ.qregs[0][:]) # qargs = estimate + row_norm + vector
    circ.append(QPE_circ, qargs= est_reg[:] + circ.qregs[0][::-1] + row_norm_reg[::-1])
    
#     #####
#     vector_creg = ClassicalRegister(np.log2(n),name='vectormeasure')
#     row_norm_creg = ClassicalRegister(np.log2(m),name='normeasure')
#     est_creg= ClassicalRegister(est_qubits,name='estmeasure')
#     circ.add_register(vector_creg,row_norm_creg,est_creg)
#     circ.measure(circ.qregs[0],vector_creg)
#     circ.measure(row_norm_reg,row_norm_creg)
#     circ.measure(est_reg,est_creg)
#     backend = qiskit.Aer.get_backend('qasm_simulator')
#     job = execute(circ,backend,shots=10000,seed_transpilier=0,seed_simulator=0)
#     counts = job.result().get_counts()
#     print(counts)
#     print(1/0)
#     #####
    
    
    # 4. Compute sigma_i = cos(theta_i / 2)||A|| and uncompute output of phase estimation
    vector_creg = ClassicalRegister(np.log2(n),name='vectermeasure') # measure vector_reg (logn)
#    row_norm_creg = ClassicalRegister(np.log2(m),name='normmeasure')
    est_creg = ClassicalRegister(est_qubits,name='estmeasure')
    circ.add_register(vector_creg, est_creg)
    
    circ.measure(est_reg, est_creg)
    circ.measure(circ.qregs[0][:], vector_creg)
    
    
    backend = qiskit.Aer.get_backend('qasm_simulator')
    job = execute(circ,backend,shots=10000,seed_transpilier=0,seed_simulator=0)
    counts = job.result().get_counts()
    print(counts)
    sv_dict = compute_SVE(counts,est_qubits) # {vector: sv_in_binary}
    
    for vector,sv_in_bin in sv_dict.items():
        temp = QuantumCircuit(est_qubits)
        for idx,bit in enumerate(sv_in_bin):
            if bit == '1':
                temp.x(idx)
        
        sv_gate = temp.to_gate(label=sv_in_bin).control(len(circ.qregs[0]), ctrl_state=vector[::-1]) # Controlled on vector register
        sv_circ.append(sv_gate, qargs = sv_circ.qregs[0][:] + est_reg[:])

    # 5. Apply inverse of step 2 to obtain sum x_i |v_i> |sigma_i>
    Q_inv = Q_gate.inverse()
    Q_inv.label = 'Q inv'
    sv_circ.append(Q_inv, qargs= circ.qregs[0][::-1] + row_norm_reg[::-1])
#    print(sv_circ.draw())
    
    return sv_circ

In [131]:
def quantum_projection(x,A,threshold):
    """
    Project the vector x onto the subspace spanned by singular vectors with corresponding singular values
    above the threshold (See Algorithm 5.2)
    
    Args:
        x: np.array (n)
        A: np.array (m x n)
        threshold: (float,float) (σ,κ)
    
    Return:
        A quantum circuit that performs the projection
    """
    m,n = A.shape[0],A.shape[1]
    assert len(x) == n, "Dim 0 of x should match dim 1 of A"
    
    vector_reg = QuantumRegister(np.log2(n),name='vector')
    circuit = QuantumCircuit(vector_reg)
    
    # 1. Initialize data state (CHECKED)
    init_circ = amplitude_encoding(x)
    circuit.append(init_circ, vector_reg)
    
    sigma, kappa = threshold
    precision = (kappa/2.) * (sigma / np.linalg.norm(A))
    
    # 2. Apply SVE on |x> to obtain sum x_i |0^logm> |v_i> |sigma_i>
    est_qubits = int(np.ceil(np.log2(np.pi / precision)))
    circuit = quantum_SVE(circuit,x,A,est_qubits)
    
    # 3. Map |t>|0> to |t>|1> if t < sigma*(1 - kappa/2)
    est_upper = 2**est_qubits
    ctrl_threshold = sigma*(1-kappa/2) / np.linalg.norm(A) # Divide by |A| to compensate for sigma_i computation
    print(ctrl_threshold)
    
    sv_bound = lambda sv: min(sv,est_upper-1)
#    sv_to_bin_sv = lambda sv: bin(int(np.round(sv,decimals=0)))[2:].zfill(est_qubits) # Encode a singular value as the nearest bitstring
    #sv_true_to_sv = lambda sv_true: (sv_true + 1) * (est_upper) / 2 # +1, then *2^n / 2 to map [-1,1] to [0,2] to bitstring(n)
    sv_true_to_sv = lambda sv_true: sv_true * est_upper
    ctrl_threshold = sv_bound(sv_true_to_sv(ctrl_threshold))
    
    print("ctrl_threshold = {}".format(ctrl_threshold))
    upper_ctrl_state = int(np.floor(ctrl_threshold))
    if upper_ctrl_state == ctrl_threshold:
        upper_ctrl_state -= 1
    ctrl_states = range(upper_ctrl_state + 1)
    
    print("ctrl_states = {}".format(ctrl_states))
    
    ancilla_reg = QuantumRegister(1,name='ancilla')
    circuit.add_register(ancilla_reg)
    
    vector_reg = circuit.qregs[0]
    row_norm_reg = circuit.qregs[1]
    est_reg = circuit.qregs[2]
    
    X_gate = qiskit.circuit.library.XGate("X")
    for ctrl_state in ctrl_states:
        circuit.append(X_gate.control(est_qubits, ctrl_state=ctrl_state), est_reg[:] + ancilla_reg[:])
        
    # 4. (skip, no need to uncompute SV, just directly measure vector and ancilla)
    # 5. If ancilla = 0, output vector; otherwise, repeat from step 1
    
    vector_creg = ClassicalRegister(np.log2(n),name='vectormeasure')
    ancilla_creg = ClassicalRegister(1,name='ancillameasure')    
    circuit.add_register(vector_creg,ancilla_creg)
    
    circuit.measure(ancilla_reg, ancilla_creg)
    circuit.measure(vector_reg, vector_creg)
    
    print(circuit.draw())
    
    backend = qiskit.Aer.get_backend('qasm_simulator')
    job = execute(circuit,backend,shots=10000,seed_transpilier=0,seed_simulator=0)
    counts = job.result().get_counts()
    print(counts)
    
    return counts
    

In [133]:
x = np.array([0,1,2,0])
A = np.array([[1,1,0,0],[5,6,7,0]])
#x = np.array([1,0.1,0.1,0.1])
#A = np.array([[1.25, 0, 0, 0], [0, 1.25, 0, 0]])
#A = np.array([[1.25,0,0,1.25],[0,1.25,1.25,0]])
#A = np.array([[1.25, 1.25, 0, 1.25],[0, 0, 1.25, 0],[1.25, 1.25, 0, 0],[0, 0, 1.25,0]])

eps = 0.1
p = 0.8
k = 2
sigma = np.sqrt((eps**2)*p/(2*k)) * np.linalg.norm(A)
kappa = 1/3
threshold = (sigma,kappa)
counts = quantum_projection(x,A,threshold)

{'000000000 01': 1, '001000000 01': 1, '001000001 00': 1, '001000001 10': 1, '001001000 01': 1, '001001001 01': 1, '001010000 01': 2, '001010001 00': 3, '001010001 01': 2, '001010001 10': 1, '001010110 00': 1, '001011110 00': 19, '001011110 01': 9, '001011110 10': 6, '001011111 01': 5, '001011111 10': 1, '001100000 00': 1, '001100000 01': 1, '001100001 00': 13, '001100001 01': 7, '001100001 10': 12, '001101010 00': 1, '001101110 00': 2, '001101110 01': 6, '001101110 10': 3, '001101111 10': 1, '001110001 00': 1, '001110001 01': 2, '001110010 01': 1, '001110101 10': 1, '001111110 00': 1, '001111110 01': 1, '001111110 10': 1, '010000001 00': 2, '010000001 10': 2, '000001000 01': 1, '010001001 10': 1, '010001110 00': 1, '010001110 01': 1, '010010000 00': 1, '010010000 01': 2, '010010000 10': 1, '010010001 00': 5, '010010001 01': 6, '010010001 10': 8, '010011001 00': 1, '010011010 10': 1, '010011110 00': 311, '010011110 01': 223, '010011110 10': 238, '010011111 00': 6, '010011111 01': 63, '

{'0 01': 2041, '1 10': 7959}


In [134]:
recc_list = []
shots = sum(counts.values())
for state,freq in counts.items():
    anc,item = state.split()
    if anc == '0':
        recc_list.append(int(item,2)+1)
print("Recommendation list: {}".format(recc_list))

Recommendation list: [2]
